In [81]:
import speech_recognition as sr
import os
import asyncio
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv, find_dotenv
import edge_tts

In [82]:
def ouvirMic():
  # habilitar mic
  microfone = sr.Recognizer()
  print("Diga alguma coisa: ")
  with sr.Microphone() as source:
    # armazena o audio em texto
    audio = microfone.listen(source)
  try:
    frase = microfone.recognize_google(audio, language="pt-BR")
    return frase
  except sr.UnknownValueError:
    print("Não entendi")
  return False

In [83]:

load_dotenv()
key = os.getenv("API_GROQ")

In [84]:
template = """
Você é um assistente e eu sou o seu mestre. Você é designado a me ajudar para solucionar qualquer questão lógica que eu fizer. Sempre me trate e se refira a mim como "Mestre": {text}
"""

In [85]:
prompt = PromptTemplate.from_template(template=template)
chat = ChatGroq(api_key=key , model='llama-3.1-8b-instant')
chain = prompt | chat

In [86]:
def Translate():
  text = ouvirMic()
  translate = f"{chain.invoke(text).content}"
  return translate
  
translate = Translate()

Diga alguma coisa: 


In [87]:
VOICES = ['pt-BR-AntonioNeural']
VOICE = VOICES[0]
OUTPUT_FILE = 'text_speech.mp3'

In [88]:
async def Voice() -> None:
  communicate = edge_tts.Communicate(translate, VOICE)
  await communicate.save(OUTPUT_FILE)

In [89]:
if __name__ == "__main__":
  loop = asyncio.get_event_loop()
  try:
    loop.run_until_complete(await Voice())
    os.system('text_speech.mp3')
  finally:
    loop.close()

RuntimeError: Cannot close a running event loop